<a href="https://colab.research.google.com/github/aarish7771/Deep_Learning/blob/main/AutoEncoder_for_Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [2]:
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin1')

In [3]:
from IPython.display import display
display(movies.head())
display(users.head())
display(ratings.head())

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Creating the training set and the test set

In [4]:
training_set = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies

In [5]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
print(nb_users)
print(nb_movies)

943
1682


## Converting the data into an array with users in rows and movies in columns

In [6]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[data[:, 0] == id_users, 1]
    id_ratings = data[data[:, 0] == id_users, 2]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data to torch tensors

In [7]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the Architecture of Neural Network

In [8]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 25)
    self.fc2 = nn.Linear(25, 14)
    self.fc3 = nn.Linear(14, 25)
    self.fc4 = nn.Linear(25, nb_movies)
    self.activation = nn.Sigmoid()

  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE

In [9]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('Epoch: '+str(epoch)+'Loss: '+str(train_loss/s))

Epoch: 1Loss: tensor(1.6814)
Epoch: 2Loss: tensor(1.0826)
Epoch: 3Loss: tensor(1.0495)
Epoch: 4Loss: tensor(1.0381)
Epoch: 5Loss: tensor(1.0328)
Epoch: 6Loss: tensor(1.0294)
Epoch: 7Loss: tensor(1.0277)
Epoch: 8Loss: tensor(1.0261)
Epoch: 9Loss: tensor(1.0254)
Epoch: 10Loss: tensor(1.0247)
Epoch: 11Loss: tensor(1.0238)
Epoch: 12Loss: tensor(1.0237)
Epoch: 13Loss: tensor(1.0231)
Epoch: 14Loss: tensor(1.0228)
Epoch: 15Loss: tensor(1.0229)
Epoch: 16Loss: tensor(1.0223)
Epoch: 17Loss: tensor(1.0224)
Epoch: 18Loss: tensor(1.0222)
Epoch: 19Loss: tensor(1.0220)
Epoch: 20Loss: tensor(1.0210)
Epoch: 21Loss: tensor(1.0183)
Epoch: 22Loss: tensor(1.0168)
Epoch: 23Loss: tensor(1.0138)
Epoch: 24Loss: tensor(1.0146)
Epoch: 25Loss: tensor(1.0098)
Epoch: 26Loss: tensor(1.0108)
Epoch: 27Loss: tensor(1.0067)
Epoch: 28Loss: tensor(1.0062)
Epoch: 29Loss: tensor(1.0033)
Epoch: 30Loss: tensor(1.0041)
Epoch: 31Loss: tensor(1.0005)
Epoch: 32Loss: tensor(0.9996)
Epoch: 33Loss: tensor(0.9964)
Epoch: 34Loss: tens

## Testing the SAE

In [13]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user])
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[0][target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('Test Loss: '+str(test_loss/s))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: tensor(0.9519)
